In [1]:
!pip install -q pypdf
!pip install -q python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 9.9 MB/s eta 0:00:00


In [2]:
!pip install -q llama-index
!pip install -q llama-index-llms-huggingface
!pip install -q llama-index-embeddings-langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 68.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.14.0 requires huggingface-hu

In [3]:
import os, logging, sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
from huggingface_hub import login

login(token="hf token")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


download pdf sebagai sumber data

In [5]:
!curl https://www.bcartscouncil.ca/app/uploads/sites/508/2022/05/FY2022-23-Application-Assistance-Guidelines-V3-1.pdf --output dataset.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  145k  100  145k    0     0  83585      0  0:00:01  0:00:01 --:--:-- 83598


ekstrak pdf menjadi teks

In [6]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_dir="/content/", required_exts=".pdf").load_data()
documents[0]

Document(id_='3ba20b73-561b-4c41-8ec7-6b8ec47d2292', embedding=None, metadata={'page_label': '1', 'file_name': 'dataset.pdf', 'file_path': '/content/dataset.pdf', 'file_type': 'application/pdf', 'file_size': 148806, 'creation_date': '2024-12-27', 'last_modified_date': '2024-12-27'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Application Assistance Program Guidelines \nCurrent as of: May 18, 2022 \nThese guidelines are revised and updated with regularity. Ensure you have the most current version of \nthese guidelines by checking the website at this link.  \n \nIntroduction \nThe BC Arts Council welcomes applications from and funds proj

buat vektor embedding menggunakan HuggingFaceEmbeddings.

HuggingFaceEmbeddings merupakan wrapper library LangChain yang akan generating text embeddings. sehingga kita bisa menggunakan sentence embedding model yang ada di huggingface seperti untuk task semantic search, document clustering, dan question answering.

untuk task Q&A kita gunakan model `multi-qa-MiniLM-L6-cos-v1`

[Local Embeddings with HuggingFace](https://docs.llamaindex.ai/en/stable/examples/embeddings/huggingface/)

In [7]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.6/915.6 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.3.1
    Uninstalling sentence-transformers-3.3.1:
      Successfully uninstalled sentence-transformers-3.3.1


In [8]:
# from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

EMBEDDING_MODEL_NAME = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"

embed_model = HuggingFaceEmbedding(model_name=EMBEDDING_MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector Store Index

Vector Store adalah suatu jenis index penyimpanan data dimana data tersimpan sebagai vector embedding. vector embedding ini merupakan angka yang merepresentasikan makna dari data secara semantik. teknik ini efisien untuk tugas similarity search, dimana item yang paling mirip dengan query (data yang dicari) akan diambil (diberikan sebagai hasil dari pencarian)

In [9]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, embed_model = embed_model)

set up prompt

In [33]:
#from llama_index.core.prompts.prompts import PromptTemplate
from llama_index.core import PromptTemplate

system_prompt = """<|SYSTEM|># You are an AI-enabled admin assistant.
Your goal is to answer questions accurately using only the context provided.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

# LLM_MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf" # model terlalu besar
LLM_MODEL_NAME = "google/flan-t5-base"

In [42]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM # Import AutoModelForSeq2SeqLM
from llama_index.llms.huggingface import HuggingFaceLLM

# To import models from HuggingFace directly
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=512,
    generate_kwargs={"temperature": 0.1, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=LLM_MODEL_NAME,
    model_name=LLM_MODEL_NAME,
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True},
    # Use AutoModelForSeq2SeqLM for T5 models
    model = AutoModelForSeq2SeqLM.from_pretrained(LLM_MODEL_NAME,
                                                  device_map="auto",
                                                  ), # Pass model_kwargs here
    tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME) # Initialize tokenizer
)

In [ ]:
# import torch
# from llama_index.llms.huggingface import HuggingFaceLLM

# # To import models from HuggingFace directly
# llm = HuggingFaceLLM(
#     context_window=4096,
#     max_new_tokens=512,
#     generate_kwargs={"temperature": 0.1, "do_sample": True},
#     system_prompt=system_prompt,
#     query_wrapper_prompt=query_wrapper_prompt,
#     tokenizer_name=LLM_MODEL_NAME,
#     model_name=LLM_MODEL_NAME,
#     device_map="auto",
#     # uncomment this if using CUDA to reduce memory usage
#     # model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
# )

In [48]:
from llama_index.core import Settings

Settings.embed_model = embed_model
Settings.llm = llm
Settings.chunk_size = 1024 # terlalu besar
# Settings.chunk_size = 512
#Settings.chunk_overlap = 256

inisialisasi query engine.

cari 5 response yang paling mirip sesuai dengan prompt yang diberikan

In [49]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)

format output

In [50]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

Generate contextual results menggunakan retrieval-augmented prompt

In [52]:
query_engine.query("Tell me about the text")

Response(response='Empty Response', source_nodes=[NodeWithScore(node=TextNode(id_='a4d5a0f8-fcac-4ec1-b1d4-56890c55eb3f', embedding=None, metadata={'page_label': '5', 'file_name': 'dataset.pdf', 'file_path': '/content/dataset.pdf', 'file_type': 'application/pdf', 'file_size': 148806, 'creation_date': '2024-12-27', 'last_modified_date': '2024-12-27'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a7c0d01f-6c77-4568-b3eb-b5ebfb51ef90', node_type='4', metadata={'page_label': '5', 'file_name': 'dataset.pdf', 'file_path': '/content/dataset.pdf', 'file_type': 'application/pdf', 'file_size': 148806, 'creation_date': '2024-12-27', 'last_modified_date': '2024-12-27'}, hash='7afb7df0b1c5704b2c4f28fc9dc44667f8c5d397

In [53]:
done = False
while not done:
  print("*"*30)
  question = input("Enter your question: ")
  response = query_engine.query(question)
  print(response)
  done = input("End the chat? (y/n): ") == "y"

******************************
Enter your question: what the title of text ?
Empty Response
End the chat? (y/n): y


sumber

https://ai-journey.com/2024/05/build-rag-chatbot-using-pdf-documents-powered-by-huggingface-and-llamaindex/